# Comparing Different Architecture Performances

In [6]:
#Consulted ChatGPT to write / debug this code 

## SVD

In [2]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
import pandas as pd

# Load data from CSV file into a DataFrame
file_path =  '/Users/kaileyfitzgerald/Desktop/ML and Data Sci/final projrct/interactions_small.csv'  # Replace 'path_to_your_file.csv' with your file path
ratings_data = pd.read_csv(file_path)

# Create a Surprise Reader and Dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_data[['user_id', 'book_id', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Use SVD algorithm (you can experiment with other algorithms as well)
model = SVD()

# Fit the model on the train set
model.fit(trainset)

# Test the model on the test set
predictions = model.test(testset)

# Calculate RMSE (Root Mean Squared Error)
accuracy = rmse(predictions)
print(f"RMSE on test set: {accuracy:.4f}")


RMSE: 1.7536
RMSE on test set: 1.7536


## RNN

In [3]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming you have loaded your data into the following DataFrame
# with columns: 'user_id', 'book_id', 'rating'
ratings_data = pd.read_csv(file_path)

# Creating mappings for user_id and book_id
user_ids = ratings_data['user_id'].unique()
book_ids = ratings_data['book_id'].unique()
num_users = len(user_ids)
num_books = len(book_ids)

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
book_to_idx = {book_id: idx for idx, book_id in enumerate(book_ids)}

ratings_data['user_idx'] = ratings_data['user_id'].map(user_to_idx)
ratings_data['book_idx'] = ratings_data['book_id'].map(book_to_idx)

# Splitting data into train, validation, and test sets
train_data, test_data = train_test_split(ratings_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data[['user_idx', 'book_idx', 'rating']].values.astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Instantiate datasets and DataLoaders for train, validation, and test sets
train_dataset = RatingDataset(train_data)
val_dataset = RatingDataset(val_data)
test_dataset = RatingDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the RNN model
class RNNRecommender(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim=32, hidden_dim=64):
        super(RNNRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.book_embedding = nn.Embedding(num_books, embedding_dim)
        self.rnn = nn.RNN(embedding_dim * 2, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, user_idx, book_idx):
        user_embed = self.user_embedding(user_idx)
        book_embed = self.book_embedding(book_idx)
        combined = torch.cat((user_embed, book_embed), dim=1)
        output, _ = self.rnn(combined.unsqueeze(1))
        output = self.fc(output.squeeze())
        return output

# Instantiate the model, define loss function and optimizer
model = RNNRecommender(num_users, num_books)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        user_idx, book_idx, ratings = batch[:, 0], batch[:, 1], batch[:, 2].float()

        optimizer.zero_grad()
        predictions = model(user_idx, book_idx).squeeze()
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for val_batch in val_loader:
        val_user_idx, val_book_idx, val_ratings = val_batch[:, 0], val_batch[:, 1], val_batch[:, 2].float()
        val_predictions = model(val_user_idx, val_book_idx).squeeze()
        val_loss = criterion(val_predictions, val_ratings)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {np.mean(val_losses):.4f}")

# Calculate RMSE on the test set
predictions_list = []
actual_ratings_list = []

model.eval()
for test_batch in test_loader:
    test_user_idx, test_book_idx, test_ratings = test_batch[:, 0], test_batch[:, 1], test_batch[:, 2].float()
    test_predictions = model(test_user_idx, test_book_idx).squeeze()

    predictions_list.extend(test_predictions.tolist())
    actual_ratings_list.extend(test_ratings.tolist())

# Convert lists to NumPy arrays
predictions_arr = np.array(predictions_list)
actual_ratings_arr = np.array(actual_ratings_list)

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions_arr - actual_ratings_arr) ** 2))
print(f"RMSE on the test set: {rmse:.4f}")


Epoch [1/10], Train Loss: 3.7372, Validation Loss: 4.2585
Epoch [2/10], Train Loss: 3.8863, Validation Loss: 4.0407
Epoch [3/10], Train Loss: 4.0170, Validation Loss: 3.8229
Epoch [4/10], Train Loss: 3.0864, Validation Loss: 3.6919
Epoch [5/10], Train Loss: 3.3340, Validation Loss: 3.5586
Epoch [6/10], Train Loss: 2.8904, Validation Loss: 3.5229
Epoch [7/10], Train Loss: 3.2757, Validation Loss: 3.4073
Epoch [8/10], Train Loss: 3.3052, Validation Loss: 3.4136
Epoch [9/10], Train Loss: 2.1136, Validation Loss: 3.3677
Epoch [10/10], Train Loss: 2.4426, Validation Loss: 3.3862
RMSE on the test set: 1.8112


## MLP

In [4]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming you have loaded your data into the following DataFrame
# with columns: 'user_id', 'book_id', 'rating'
ratings_data = pd.read_csv(file_path)

# Creating mappings for user_id and book_id
user_ids = ratings_data['user_id'].unique()
book_ids = ratings_data['book_id'].unique()
num_users = len(user_ids)
num_books = len(book_ids)

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
book_to_idx = {book_id: idx for idx, book_id in enumerate(book_ids)}

ratings_data['user_idx'] = ratings_data['user_id'].map(user_to_idx)
ratings_data['book_idx'] = ratings_data['book_id'].map(book_to_idx)

# Splitting data into train, validation, and test sets
train_data, test_data = train_test_split(ratings_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data[['user_idx', 'book_idx', 'rating']].values.astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Instantiate datasets and DataLoaders for train, validation, and test sets
train_dataset = RatingDataset(train_data)
val_dataset = RatingDataset(val_data)
test_dataset = RatingDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the MLP model
class MLPRecommender(nn.Module):
    def __init__(self, num_users, num_books, hidden_dim=64):
        super(MLPRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, hidden_dim)
        self.book_embedding = nn.Embedding(num_books, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, user_idx, book_idx):
        user_embed = self.user_embedding(user_idx)
        book_embed = self.book_embedding(book_idx)
        combined = torch.cat((user_embed, book_embed), dim=1)
        x = self.relu(self.fc1(combined))
        x = self.relu(self.fc2(x))
        output = self.fc3(x)
        return output

# Instantiate the model, define loss function and optimizer
model = MLPRecommender(num_users, num_books)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        user_idx, book_idx, ratings = batch[:, 0], batch[:, 1], batch[:, 2].float()

        optimizer.zero_grad()
        predictions = model(user_idx, book_idx).squeeze()
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for val_batch in val_loader:
        val_user_idx, val_book_idx, val_ratings = val_batch[:, 0], val_batch[:, 1], val_batch[:, 2].float()
        val_predictions = model(val_user_idx, val_book_idx).squeeze()
        val_loss = criterion(val_predictions, val_ratings)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {np.mean(val_losses):.4f}")

# Calculate RMSE on the test set
predictions_list = []
actual_ratings_list = []

model.eval()
for test_batch in test_loader:
    test_user_idx, test_book_idx, test_ratings = test_batch[:, 0], test_batch[:, 1], test_batch[:, 2].float()
    test_predictions = model(test_user_idx, test_book_idx).squeeze()

    predictions_list.extend(test_predictions.tolist())
    actual_ratings_list.extend(test_ratings.tolist())

# Convert lists to NumPy arrays
predictions_arr = np.array(predictions_list)
actual_ratings_arr = np.array(actual_ratings_list)

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions_arr - actual_ratings_arr) ** 2))
print(f"RMSE on the test set: {rmse:.4f}")


Epoch [1/10], Train Loss: 3.5365, Validation Loss: 3.9620
Epoch [2/10], Train Loss: 3.9773, Validation Loss: 3.6732
Epoch [3/10], Train Loss: 2.6881, Validation Loss: 3.5180
Epoch [4/10], Train Loss: 2.1160, Validation Loss: 3.5500
Epoch [5/10], Train Loss: 2.7185, Validation Loss: 3.6012
Epoch [6/10], Train Loss: 1.9694, Validation Loss: 3.7021
Epoch [7/10], Train Loss: 1.5401, Validation Loss: 3.8142
Epoch [8/10], Train Loss: 1.2880, Validation Loss: 3.9416
Epoch [9/10], Train Loss: 0.7933, Validation Loss: 4.0848
Epoch [10/10], Train Loss: 0.6287, Validation Loss: 4.1794
RMSE on the test set: 2.0178


## LSTM

In [5]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming you have loaded your data into the following DataFrame
# with columns: 'user_id', 'book_id', 'rating'
ratings_data = pd.read_csv(file_path)

# Creating mappings for user_id and book_id
user_ids = ratings_data['user_id'].unique()
book_ids = ratings_data['book_id'].unique()
num_users = len(user_ids)
num_books = len(book_ids)

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
book_to_idx = {book_id: idx for idx, book_id in enumerate(book_ids)}

ratings_data['user_idx'] = ratings_data['user_id'].map(user_to_idx)
ratings_data['book_idx'] = ratings_data['book_id'].map(book_to_idx)

# Splitting data into train, validation, and test sets
train_data, test_data = train_test_split(ratings_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data[['user_idx', 'book_idx', 'rating']].values.astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Instantiate datasets and DataLoaders for train, validation, and test sets
train_dataset = RatingDataset(train_data)
val_dataset = RatingDataset(val_data)
test_dataset = RatingDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the LSTM model
class LSTMRecommender(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim=32, hidden_dim=64):
        super(LSTMRecommender, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.book_embedding = nn.Embedding(num_books, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, user_idx, book_idx):
        user_embed = self.user_embedding(user_idx)
        book_embed = self.book_embedding(book_idx)
        combined = torch.cat((user_embed, book_embed), dim=1)
        output, _ = self.lstm(combined.unsqueeze(1))
        output = self.fc(output.squeeze())
        return output

# Instantiate the model, define loss function and optimizer
model = LSTMRecommender(num_users, num_books)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        user_idx, book_idx, ratings = batch[:, 0], batch[:, 1], batch[:, 2].float()

        optimizer.zero_grad()
        predictions = model(user_idx, book_idx).squeeze()
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for val_batch in val_loader:
        val_user_idx, val_book_idx, val_ratings = val_batch[:, 0], val_batch[:, 1], val_batch[:, 2].float()
        val_predictions = model(val_user_idx, val_book_idx).squeeze()
        val_loss = criterion(val_predictions, val_ratings)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {np.mean(val_losses):.4f}")

# Calculate RMSE on the test set
predictions_list = []
actual_ratings_list = []

model.eval()
for test_batch in test_loader:
    test_user_idx, test_book_idx, test_ratings = test_batch[:, 0], test_batch[:, 1], test_batch[:, 2].float()
    test_predictions = model(test_user_idx, test_book_idx).squeeze()

    predictions_list.extend(test_predictions.tolist())
    actual_ratings_list.extend(test_ratings.tolist())

# Convert lists to NumPy arrays
predictions_arr = np.array(predictions_list)
actual_ratings_arr = np.array(actual_ratings_list)

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions_arr - actual_ratings_arr) ** 2))
print(f"RMSE on the test set: {rmse:.4f}")


Epoch [1/10], Train Loss: 3.3112, Validation Loss: 4.2826
Epoch [2/10], Train Loss: 2.7776, Validation Loss: 3.9669
Epoch [3/10], Train Loss: 3.8131, Validation Loss: 3.7371
Epoch [4/10], Train Loss: 3.5775, Validation Loss: 3.6085
Epoch [5/10], Train Loss: 3.1538, Validation Loss: 3.4857
Epoch [6/10], Train Loss: 2.9954, Validation Loss: 3.4582
Epoch [7/10], Train Loss: 2.5588, Validation Loss: 3.4439
Epoch [8/10], Train Loss: 1.9476, Validation Loss: 3.4658
Epoch [9/10], Train Loss: 1.6697, Validation Loss: 3.4890
Epoch [10/10], Train Loss: 1.7356, Validation Loss: 3.5240
RMSE on the test set: 1.8509


In [54]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming you have loaded your data into the following DataFrame
# with columns: 'user_id', 'book_id', 'rating'
ratings_data = pd.read_csv(file_path)

# Creating mappings for user_id and book_id
user_ids = ratings_data['user_id'].unique()
book_ids = ratings_data['book_id'].unique()
num_users = len(user_ids)
num_books = len(book_ids)

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
book_to_idx = {book_id: idx for idx, book_id in enumerate(book_ids)}

ratings_data['user_idx'] = ratings_data['user_id'].map(user_to_idx)
ratings_data['book_idx'] = ratings_data['book_id'].map(book_to_idx)

# Splitting data into train, validation, and test sets
train_data, test_data = train_test_split(ratings_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data[['user_idx', 'book_idx', 'rating']].values.astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Instantiate datasets and DataLoaders for train, validation, and test sets
train_dataset = RatingDataset(train_data)
val_dataset = RatingDataset(val_data)
test_dataset = RatingDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the RNN model with embeddings
class RNNWithEmbeddings(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim=32, hidden_dim=64):
        super(RNNWithEmbeddings, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.book_embeddings = nn.Embedding(num_books, embedding_dim)
        self.rnn = nn.RNN(embedding_dim * 2, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, user_idx, book_idx):
        user_embed = self.user_embeddings(user_idx)
        book_embed = self.book_embeddings(book_idx)
        combined = torch.cat((user_embed, book_embed), dim=1)
        output, _ = self.rnn(combined.unsqueeze(1))
        output = self.fc(output.squeeze())
        return output

# Instantiate the model, define loss function and optimizer
model = RNNWithEmbeddings(num_users, num_books)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        user_idx, book_idx, ratings = batch[:, 0], batch[:, 1], batch[:, 2].float()

        optimizer.zero_grad()
        predictions = model(user_idx, book_idx).squeeze()
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for val_batch in val_loader:
        val_user_idx, val_book_idx, val_ratings = val_batch[:, 0], val_batch[:, 1], val_batch[:, 2].float()
        val_predictions = model(val_user_idx, val_book_idx).squeeze()
        val_loss = criterion(val_predictions, val_ratings)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {np.mean(val_losses):.4f}")

# Calculate RMSE on the test set
predictions_list = []
actual_ratings_list = []

model.eval()
for test_batch in test_loader:
    test_user_idx, test_book_idx, test_ratings = test_batch[:, 0], test_batch[:, 1], test_batch[:, 2].float()
    test_predictions = model(test_user_idx, test_book_idx).squeeze()

    predictions_list.extend(test_predictions.tolist())
    actual_ratings_list.extend(test_ratings.tolist())

# Convert lists to NumPy arrays
predictions_arr = np.array(predictions_list)
actual_ratings_arr = np.array(actual_ratings_list)

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions_arr - actual_ratings_arr) ** 2))
print(f"RMSE on the test set: {rmse:.4f}")


Epoch [1/10], Train Loss: 4.6175, Validation Loss: 4.3145
Epoch [2/10], Train Loss: 3.5785, Validation Loss: 4.0870
Epoch [3/10], Train Loss: 3.3141, Validation Loss: 3.9002
Epoch [4/10], Train Loss: 3.4085, Validation Loss: 3.7357
Epoch [5/10], Train Loss: 4.1850, Validation Loss: 3.5975
Epoch [6/10], Train Loss: 3.1434, Validation Loss: 3.5376
Epoch [7/10], Train Loss: 3.1276, Validation Loss: 3.4495
Epoch [8/10], Train Loss: 2.5021, Validation Loss: 3.4983
Epoch [9/10], Train Loss: 1.8358, Validation Loss: 3.4359
Epoch [10/10], Train Loss: 2.2710, Validation Loss: 3.3932
RMSE on the test set: 1.8302


## MLP with Embedding

In [55]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming you have loaded your data into the following DataFrame
# with columns: 'user_id', 'book_id', 'rating'
ratings_data = pd.read_csv(file_path)

# Creating mappings for user_id and book_id
user_ids = ratings_data['user_id'].unique()
book_ids = ratings_data['book_id'].unique()
num_users = len(user_ids)
num_books = len(book_ids)

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
book_to_idx = {book_id: idx for idx, book_id in enumerate(book_ids)}

ratings_data['user_idx'] = ratings_data['user_id'].map(user_to_idx)
ratings_data['book_idx'] = ratings_data['book_id'].map(book_to_idx)

# Splitting data into train, validation, and test sets
train_data, test_data = train_test_split(ratings_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Define a PyTorch Dataset
class RatingDataset(Dataset):
    def __init__(self, data):
        self.data = data[['user_idx', 'book_idx', 'rating']].values.astype(np.int64)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Instantiate datasets and DataLoaders for train, validation, and test sets
train_dataset = RatingDataset(train_data)
val_dataset = RatingDataset(val_data)
test_dataset = RatingDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Define the MLP model with embeddings
class MLPWithEmbeddings(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim=32, hidden_dim=64):
        super(MLPWithEmbeddings, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.book_embeddings = nn.Embedding(num_books, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, user_idx, book_idx):
        user_embed = self.user_embeddings(user_idx)
        book_embed = self.book_embeddings(book_idx)
        combined = torch.cat((user_embed, book_embed), dim=1)
        x = self.relu(self.fc1(combined))
        x = self.relu(self.fc2(x))
        output = self.fc3(x)
        return output

# Instantiate the model, define loss function and optimizer
model = MLPWithEmbeddings(num_users, num_books)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        user_idx, book_idx, ratings = batch[:, 0], batch[:, 1], batch[:, 2].float()

        optimizer.zero_grad()
        predictions = model(user_idx, book_idx).squeeze()
        loss = criterion(predictions, ratings)
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for val_batch in val_loader:
        val_user_idx, val_book_idx, val_ratings = val_batch[:, 0], val_batch[:, 1], val_batch[:, 2].float()
        val_predictions = model(val_user_idx, val_book_idx).squeeze()
        val_loss = criterion(val_predictions, val_ratings)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {np.mean(val_losses):.4f}")

# Calculate RMSE on the test set
predictions_list = []
actual_ratings_list = []

model.eval()
for test_batch in test_loader:
    test_user_idx, test_book_idx, test_ratings = test_batch[:, 0], test_batch[:, 1], test_batch[:, 2].float()
    test_predictions = model(test_user_idx, test_book_idx).squeeze()

    predictions_list.extend(test_predictions.tolist())
    actual_ratings_list.extend(test_ratings.tolist())

# Convert lists to NumPy arrays
predictions_arr = np.array(predictions_list)
actual_ratings_arr = np.array(actual_ratings_list)

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions_arr - actual_ratings_arr) ** 2))
print(f"RMSE on the test set: {rmse:.4f}")


Epoch [1/10], Train Loss: 3.9311, Validation Loss: 4.1931
Epoch [2/10], Train Loss: 4.2026, Validation Loss: 3.8984
Epoch [3/10], Train Loss: 2.4567, Validation Loss: 3.7429
Epoch [4/10], Train Loss: 2.8310, Validation Loss: 3.6405
Epoch [5/10], Train Loss: 2.0208, Validation Loss: 3.6170
Epoch [6/10], Train Loss: 2.5099, Validation Loss: 3.6965
Epoch [7/10], Train Loss: 2.3580, Validation Loss: 3.6498
Epoch [8/10], Train Loss: 2.2615, Validation Loss: 3.8220
Epoch [9/10], Train Loss: 1.9964, Validation Loss: 3.9691
Epoch [10/10], Train Loss: 1.9048, Validation Loss: 3.8894
RMSE on the test set: 1.9353
